### Run cell below to play Boxy!
##### Requires PyGame Module

In [1]:
import pygame

pygame.init()

# Standard modules
import numpy as np
#from time import sleep

# Custom classes and functions
from Super_Classes import Body, Shape
from Status import Status
from Player import Player
from Constants import display_size
from Level import Level
from Make_Sounds import ouch_sound, thud_sound
from Keyboard import Keyboard


# Initialize game
gameDisplay = pygame.display.set_mode(display_size)
pygame.display.set_caption('Boxy!')
sky = (205, 230, 247)
clock = pygame.time.Clock()



#screen2 = Body(np.array(display_size)/2,np.array(display_size)/2,corporeal=True,solid=False,velocity=[0,0])   
#screen2.shapes = [Shape(screen.self_shape(),color = None,line_color = (0,0,0))]

# Object holding all level Bodies and scenery
level = Level(num=0) # right now level 0


# Object to determine what to draw on screen (anything non overlapping isn't drawn)
screen = Body([level.player_start[0],display_size[1]/2],np.array(display_size)/2,corporeal=True,solid=False,velocity=[0,0])   


#level.background_list.append(screen2)

death_delay = 60 # time after player's death that the level keeps tickin'
crashed = death_delay # start with a completed crash cycle to initalize everything

# initialize keyboard object to take in control data and feed it to the player object
keyboard = Keyboard()

while True: # play the game

##############################################################################
    # Handle player death and level reset
    
    if crashed or character.pos[1]>(1.5*display_size[1]): # player off screen = dead!
        if crashed == death_delay:
            character = Player(level.player_start)
            crashed = 0
            level.reset()
            character.current_status = Status(lives = 14, fruit = 85, boxes = 0) # arbitrary start for now
            keyboard.reset()
        else:    
            crashed += 1
            
        if crashed == 2: # just died, play ouch sound!
            ouch_sound()


##############################################################################
    # Note keyboard inputs and store them (including killing player by quitting)
    
    keyboard.handle_in_game_events(pygame.event.get())
      
    if keyboard.crashed and not crashed: # if q key pressed, character is dead
        crashed = 1

##############################################################################    
    # Move every non-player object and resolve their interactions
    
    level.move_objects()
    

##############################################################################
    # [if player is alive] Evolve its state, move it, and resolve interactions with other bodies

    if not crashed: # if player is dead, skip this part
          
        # determine player's state and velocity, then move
        character.evolve(keyboard)
        character.move()
          
        # sort box_list so that closest bodies interact first: prevents e.g. diagonally breaking boxes
        level.box_list = sorted(level.box_list, key= lambda x: sum((character.pos-x.pos)**2), reverse=False)
        
       # squeeze = [[False,False],[False,False]]
     
        # all objects interact with the player (destroy boxes, get fruit, land on platforms, etc)
        for i in level.box_list+level.platform_list+level.gettable_list:
            i.interact(character)
            
        # some of these interactions might have killed ya. Then ya dead!
        if character.protection < 0:
            crashed = 1
        
        
##############################################################################
    # Prepare canvas and draw visuals

    # fill display with sky color and set center position for screen (only overlapping objects will draw)
    gameDisplay.fill(sky)
    screen.pos = np.array([character.pos[0],display_size[1]/2]) # location of the center of the screen
   
    # move all non-corporeal objects to the foreground, draw all bodies, and clean foreground list of dead objects
    level.draw_level(gameDisplay,screen,character)

  #  sleep(0.1) # turned on for debugging
    pygame.display.update()
    clock.tick(35)

##############################################################################
    
pygame.quit()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyboardInterrupt: 

To Do:

Add spring boxes (metal/wood, one class that adds a shape of an arrow, and that's it, rest will be handled outside)

Add boom_box interactions with surrounding boxes/fruit (destroy them, no fruit gained)

Level saving

Game structure, saving, scores, etc.

Bad guys!

Edit sound files so they start quicker (doing this in finder made them unloadable)

fruit flies towards counters at the top when grabbed

### Squeezing is not currently configured at all (to kill player) (is this needed?)
#### Definitely need to fix: currently, falling non-breakable boxes do not kill player

Known bugs:
falling stacks of boxes ON a moving platform results in slight overlap (but nothing functionally problematic)

Credits:
“Sound effects obtained from zapsplat.com and freesound.org"
"Idea inspired by but unrelated to Crash Bandicoot"
